In [ ]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
import sqlite3
import sqllite_handler
import copy

### save below into py. creates dataframes!

In [ ]:
conn = sqlite3.connect('tables/texts_69_76.db')
cur = conn.cursor()

res = cur.execute("SELECT TEXT FROM transcript")
fetched = res.fetchall()
free_text_list = list(map(lambda x: x[0], fetched))

topic_model = BERTopic.load("plots/topic_model_69_76")

doc_df = pd.read_csv('tables/doc_69_76.csv')

messy_doc_topic_df = topic_model.get_document_info(free_text_list)

topic_desc_df = messy_doc_topic_df[['Name','Top_n_words']].drop_duplicates(ignore_index=True)
doc_topic_df = pd.DataFrame({'id_to_text':doc_df['id_to_text'],'assigned_topic':messy_doc_topic_df['Name']})

topic_desc_df.to_csv('tables/topic_descp_69_76.csv')
doc_topic_df.to_csv('tables/doc_topic_69_76.csv')

### construction below

In [42]:
# LDA topic modeling

import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
import spacy
import math

pyLDAvis.enable_notebook()
nlp = spacy.load('en_core_web_sm')

/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [51]:
doc_df = pd.read_csv('tables/doc_69_76.csv')
free_text_list = doc_df['text'].values
free_text_list = list(map(lambda x: ' ' if not x or (isinstance(x, float) and math.isnan(x)) else x, free_text_list))

In [52]:
processed_free_text_list = list(map(lambda row: [t.lower_ for t in nlp(row) if not t.is_punct and not t.is_stop], free_text_list))

dictionary = Dictionary(processed_free_text_list)

dictionary.filter_extremes(keep_n=10000)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in processed_free_text_list]

lda = LdaModel(corpus = doc_term_matrix, id2word=dictionary, num_topics=10)

In [ ]:
gensimvis.prepare(lda, doc_term_matrix, dictionary)

/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.009873  0.009348       1        1  22.612450
7      0.000688  0.002053       2        1  15.483750
0      0.013631 -0.006745       3        1  12.659692
8      0.032844 -0.002441       4        1  11.922527
3     -0.013982 -0.000855       5        1  10.002635
6     -0.008729  0.011338       6        1   8.874086
4     -0.007385  0.009579       7        1   8.695364
1     -0.010992 -0.017496       8        1   5.575023
2     -0.008999 -0.005180       9        1   2.569905
9     -0.006950  0.000400      10        1   1.604570, topic_info=           Term        Freq       Total Category  logprob  loglift
2938          k  255.000000  255.000000  Default  30.0000  30.0000
1456  kissinger  978.000000  978.000000  Default  29.0000  29.0000
30          mr.  805.000000  805.000000  Default  28.0000  28.0000
1294          c  113.000000  113.000000  Default  27.0000  27.0000
2239   clerides  279.000000  279.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
135    cypriots    2.748539  153.395446  Topic10  -6.1725   0.1104
2573    demirel    2.789514  162.235985  Topic10  -6.1577   0.0691
1528     soviet    2.807801  175.425549  Topic10  -6.1512  -0.0025
1524      sisco    3.063044  299.465599  Topic10  -6.0642  -0.4503
1573        aid    2.812602  275.293971  Topic10  -6.1495  -0.4514

[748 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.206449    $
0         2  0.108657    $
0         3  0.076060    $
0         4  0.184717    $
0         5  0.076060    $
...     ...       ...  ...
1037      6  0.066442  yes
1037      7  0.103816  yes
1037      8  0.049832  yes
1037      9  0.024916  yes
1037     10  0.020763  yes

[3736 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 1, 9, 4, 7, 5, 2, 3, 10])